# Domain Generation Algorithm (DGA) Detection

## Authors
 - Gorkem Batmaz (NVIDIA) [gbatmaz@nvidia.com]
 - Bhargav Suryadevara (NVIDIA) [bsuryadevara@nvidia.com]

## Development Notes
* Developed using: RAPIDS v0.12.0 and CLX v0.12
* Last tested using: RAPIDS v0.12.0 and CLX v0.12 on Jan 28, 2020

## Table of Contents
* Introduction
* Data Importing
* Data Preprocessing
* Training and Evaluation
* Inference
* Conclusion

## Introduction
[Domain Generation Algorithms](https://en.wikipedia.org/wiki/Domain_generation_algorithm) (DGAs) are used to generate domain names that can be used by the malware to communicate with the command and control servers. IP addresses and static domain names can be easily blocked, and a DGA provides an easy method to generate a large number of domain names and rotate through them to circumvent traditional block lists. We will use a type of recurrent neural network called the [Gated Recurrent Unit](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21) (GRU) for this example. The [CLX](https://github.com/rapidsai/clx) and [RAPIDS](https://rapids.ai) libraries enable users train their models with up-to-date domain names representative of both benign and DGA generated strings. Using a CLX workflow, this capability could also be used in production. This notebook provides a view into the data science workflow to create a DGA detection implementation.

In [1]:
import os
import cudf
import torch
import s3fs
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score, average_precision_score
from clx.analytics.dga_detector import DGADetector
from clx.utils.data.dataloader import DataLoader
from clx.analytics.dga_dataset import DGADataset
from cuml.preprocessing.model_selection import train_test_split

#### Download Input Dataset from S3

In [2]:
INPUT_CSV = "benign_and_dga_domains.csv"

S3_BASE_PATH = "rapidsai-data/cyber/clx"

In [3]:
# Read Benign and DGA dataset
if not os.path.exists(INPUT_CSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + INPUT_CSV, INPUT_CSV)

#### Load Input Dataset to GPU Dataframe

In [4]:
gdf = cudf.read_csv(INPUT_CSV)

In [5]:
train_data = gdf['domain']
labels = gdf['type']

Because we have only benign and DGA (malicious) categoriesm, the number of domain types need to be set to 2 (`N_DOMAIN_TYPE=2`). Vocabulary size(`CHAR_VOCAB`) is set to 128 ASCII characters. The values below set for `HIDDEN_SIZE`, `N_LAYERS` of the network, and the `LR` (Learning Rate) give an optimum balance for the network size and performance. They might need be set via experiments when working with other datasets.

In [6]:
LR = 0.001
N_LAYERS = 3
CHAR_VOCAB = 128
HIDDEN_SIZE = 100
N_DOMAIN_TYPE = 2

#### Instantiate DGA Detector
Now that the data is ready, the datasets are created, and we've set the parameters for the model, we can use the DGADetector method built into CLX to create and train the model.

In [7]:
dd = DGADetector(lr=LR)
dd.init_model(n_layers=N_LAYERS, char_vocab=CHAR_VOCAB, hidden_size=HIDDEN_SIZE, n_domain_type=N_DOMAIN_TYPE)

In [8]:
EPOCHS = 25
TRAIN_SIZE = 0.7
BATCH_SIZE = 10000
MODELS_DIR = 'models'

### Training and Evaluation
Now we train and evaluate the model.

In [9]:
%%time
dd.train_model(train_data, labels, batch_size=BATCH_SIZE, epochs=EPOCHS, train_size=0.7)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]/opt/conda/envs/rapids/lib/python3.7/site-packages/cudf/io/dlpack.py:74: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return libdlpack.to_dlpack(gdf_cols)


[100000/1433083 (7%)]	Loss: 6685.51
[200000/1433083 (14%)]	Loss: 5131.20
[300000/1433083 (21%)]	Loss: 4025.58
[400000/1433083 (28%)]	Loss: 3512.35
[500000/1433083 (35%)]	Loss: 3138.08
[600000/1433083 (42%)]	Loss: 2708.67
[700000/1433083 (49%)]	Loss: 2373.25
[800000/1433083 (56%)]	Loss: 2116.35
[900000/1433083 (63%)]	Loss: 2322.65
[1000000/1433083 (70%)]	Loss: 2222.76
[1100000/1433083 (77%)]	Loss: 2212.15
[1200000/1433083 (84%)]	Loss: 2135.32
[1300000/1433083 (91%)]	Loss: 2139.53
[1400000/1433083 (98%)]	Loss: 2171.99


Epoch:   4%|▍         | 1/25 [00:24<09:49, 24.55s/it]

Test set: Accuracy: 421091/614179 (0.6856160826078391)

[100000/1433083 (7%)]	Loss: 4163.45
[200000/1433083 (14%)]	Loss: 2837.49
[300000/1433083 (21%)]	Loss: 2089.73
[400000/1433083 (28%)]	Loss: 1823.92
[500000/1433083 (35%)]	Loss: 1620.47
[600000/1433083 (42%)]	Loss: 1390.30
[700000/1433083 (49%)]	Loss: 1217.58
[800000/1433083 (56%)]	Loss: 1085.91
[900000/1433083 (63%)]	Loss: 1040.25
[1000000/1433083 (70%)]	Loss: 1036.42
[1100000/1433083 (77%)]	Loss: 1046.11
[1200000/1433083 (84%)]	Loss: 1031.29
[1300000/1433083 (91%)]	Loss: 1066.00
[1400000/1433083 (98%)]	Loss: 1150.62


Epoch:   8%|▊         | 2/25 [00:44<08:50, 23.06s/it]

...

Test set: Accuracy: 608339/614179 (0.9904913714080097)

[100000/1433083 (7%)]	Loss: 75.46
[200000/1433083 (14%)]	Loss: 56.50
[300000/1433083 (21%)]	Loss: 48.20
[400000/1433083 (28%)]	Loss: 42.73
[500000/1433083 (35%)]	Loss: 44.61
[600000/1433083 (42%)]	Loss: 39.66
[700000/1433083 (49%)]	Loss: 36.18
[800000/1433083 (56%)]	Loss: 33.66
[900000/1433083 (63%)]	Loss: 37.19
[1000000/1433083 (70%)]	Loss: 44.64
[1100000/1433083 (77%)]	Loss: 46.17
[1200000/1433083 (84%)]	Loss: 51.50
[1300000/1433083 (91%)]	Loss: 60.99
[1400000/1433083 (98%)]	Loss: 134.19


Epoch:  96%|█████████▌| 24/25 [08:06<00:20, 20.21s/it]

Test set: Accuracy: 608413/614179 (0.9906118574552369)

[100000/1433083 (7%)]	Loss: 47.21
[200000/1433083 (14%)]	Loss: 39.97
[300000/1433083 (21%)]	Loss: 34.70
[400000/1433083 (28%)]	Loss: 31.51
[500000/1433083 (35%)]	Loss: 52.99
[600000/1433083 (42%)]	Loss: 52.56
[700000/1433083 (49%)]	Loss: 51.28
[800000/1433083 (56%)]	Loss: 49.07
[900000/1433083 (63%)]	Loss: 57.84
[1000000/1433083 (70%)]	Loss: 63.48
[1100000/1433083 (77%)]	Loss: 63.03
[1200000/1433083 (84%)]	Loss: 66.57
[1300000/1433083 (91%)]	Loss: 74.07
[1400000/1433083 (98%)]	Loss: 132.48


Epoch: 100%|██████████| 25/25 [08:26<00:00, 20.27s/it]

Test set: Accuracy: 608604/614179 (0.9909228417122695)

CPU times: user 9h 8min 3s, sys: 1h 37min 11s, total: 10h 45min 14s
Wall time: 8min 28s


### Save Model
Save pretrained model to a given output location.

In [10]:
if not os.path.exists(MODELS_DIR):
    print("Creating directory '{}'".format(MODELS_DIR))
    os.makedirs(MODELS_DIR)

now = datetime.now()
model_filename = "rnn_classifier_{}.bin".format(now.strftime("%Y-%m-%d_%H_%M_%S"))
model_filepath = os.path.join(MODELS_DIR, model_filename)
dd.save_model(model_filepath)
print("Pretrained model saved to location: '{}'".format(model_filepath))

Pretrained model saved to location: 'models/rnn_classifier_2021-01-06_20_23_15.bin'


### Inference

Using the model generated above, we now score the test dataset against the model to determine if the domain is likely generated by a DGA or benign.

In [11]:
dga_detector = DGADetector()
dga_detector.load_model(model_filepath)

domain_train, domain_test, type_train, type_test = train_test_split(gdf, "type", train_size=0.7)
test_df = cudf.DataFrame()
test_df["type"] = type_test.reset_index(drop=True)
test_df["domain"] = domain_test.reset_index(drop=True)

test_dataset = DGADataset(test_df)
test_dataloader = DataLoader(test_dataset, batchsize=BATCH_SIZE)

pred_results = []
true_results = []
for chunk in test_dataloader.get_chunks():
    pred_results.append(list(dga_detector.predict(chunk['domain']).values_host))
    true_results.append(list(chunk['type'].values_host))
pred_results = np.concatenate(pred_results)
true_results = np.concatenate(true_results)
accuracy_score = accuracy_score(pred_results, true_results)

print('Model accuracy: %s'%(accuracy_score))

Model accuracy: 0.9922400472826326


In [12]:
average_precision = average_precision_score(true_results, pred_results)

print('Average precision score: {0:0.3f}'.format(average_precision))

Average precision score: 0.979


## Conclusion

DGA detector in CLX enables users to train their models for detection and also use existing models. This capability could also be used in conjunction with log parsing efforts if the logs contain domain names. DGA detection done with CLX and RAPIDS keeps data in GPU memory, removing unnecessary copy/converts and providing a 4X speed advantage over CPU only implementations. This is esepcially true with large batch sizes.